# Data Processing using Pyspark

In [ ]:
#configuración en google colab de spark y pyspark
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
#instalar java y spark
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

In [ ]:
spark

In [ ]:
sc

In [ ]:
# Load csv Dataset 
#df=spark.read.csv('s3://<bucket/dir>/sample_data.csv',inferSchema=True,header=True)
df=spark.read.csv('/content/gdrive/MyDrive/st0263-2266/bigdata/datasets/covid19/Casos_positivos_de_COVID-19_en_Colombia-100K.csv',inferSchema=True,header=True)

In [ ]:
#columns of dataframe
df.columns

In [ ]:
#Datatypes
df.printSchema()

In [ ]:
#select only 5 columns
df.select('Nombre departamento','Edad', 'Sexo','Estado', 'Fecha de diagnóstico').show(5)

In [ ]:
#Change column name
df.select('Nombre Municipio', 'Edad').withColumnRenamed('Nombre Municipio', 'Mup').show(5)

In [ ]:
#Add a column
df.select('ID de caso', 'Fecha de recuperación', 'Edad').withColumn("Edad luego 20 años",(df["Edad"]+20)).show(10,False)

In [ ]:
#Drop columns
df.select('fecha reporte web', 'ID de caso', 'Fecha de notificación', 'Nombre departamento').drop('fecha reporte web', 'fecha de notificación').show(5)

In [ ]:
#filter the records 
df.filter(df['Nombre departamento']=='BOGOTA').show(5)

In [ ]:
# UDF
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
#using lambda function
age_udf = udf(lambda age: "Adulto" if age <= 30 else "Joven", StringType())
#apply udf on dataframe
df.withColumn("Grupo de edad", age_udf(df.Edad)).show(10,False)